
Imports and housekeeping:

For data handling we are taking in a .csv file containing our redshifted wavelengths and normalised spectral flux density re-binned into bins of 4 Angstroms. To do this we utilise pandas to convert the .csv data into a multidimensional array.

Load in Bokeh: various packages

Load in scipy.optimize for plotting the Gaussian


In [ ]:
# DATA HANDLING #

import pandas as pd
import numpy as np

# BOKEH #

# IO - allows you to display plots in a static .html and export as a .png 
# however bokeh's export functionality is dependent on selenium being installed.
from bokeh.io import output_file, output_notebook, export_png 

# Allows for the creation of figures and enables them to be called
from bokeh.plotting import figure, show
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

# Hover tool enables you to interact with the plots and pin-point x and y values 
# for any given point on the plot
from bokeh.models.tools import HoverTool

# Range1D is used to modify the ranges of X or Y values on a final plot 
# instead of using the range from the data frame
from bokeh.models import Range1d, Plot

# BOKEH INTERACTIVITY #

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# SCIPY FOR GAUSSIAN #

from scipy.optimize import curve_fit

Data Handling:

Here we load in the .csv file containing our data and convert it into a pandas data frame:

In [ ]:
# Calls in the .csv file, converts it into a pandas data frame and calls it composite_spectra
composite_spectra = pd.read_csv('combined_spectra.csv')

# Displays the top 4 lines of the data frame - this is to verify the file has been loaded in
# and stored correctly in the data frame
composite_spectra.head(4) 


Plotting the full composite spectrum:

This cell enables a full spectrum to be plotted using bokeh

The axis labels utilise unicode characters to display symbols and superscripts.


In [ ]:
# Calls upon bokeh's output_file - in this case creates a static .html file for the plot
output_file('combined_spectra.html') 


# Defines the X and Y variables by assigning a designation to column header,
# not strictly necessary but enables more concise variable naming throughout
x  = composite_spectra['wavelength']
y  = composite_spectra['flux']

p1 = figure(title = "QSO Composite Spectrum",
            x_axis_label = 'Wavelength - \u212B',
            y_axis_label = 'Spectral flux density - 10\u207B\u00B9\u2077 erg s\u207B\u00B9 cm\u207B\u00B2 \u212B\u207B\u00B9')

# Plots the spectrum as a line graph
p1.line(x, y, legend_label = "Combined spectrum", line_width = 1.5, line_color = "black") 

# Positions the legend
p1.legend.location = "top_right"

# Calls up bokeh's HoverTool, when the plot is rendered this allows you to mouse over any point and 
# displays the X and Y values of the position of your cursor and the index of the corresponding value
# in the data frame aligned vertically
p1.add_tools(HoverTool(mode='vline'))

# Shows the full spectrum, will trigger a new browser tab to open with the plot
show(p1)


Cutting the data:

This function allows you to cut the data frame into regions of interest so they can be plotted. Allows the user to specify the starting wavelength and ending wavelength of the region of interest.


In [ ]:
def cutdata(x_start, x_end, x, y):
    indx_x_start = (np.abs(x - x_start)).argmin()
    indx_x_end = (np.abs(x - x_end)).argmin()
    cut_x = x[indx_x_start:indx_x_end]
    cut_y = y[indx_x_start:indx_x_end]
    return cut_x, cut_y

Plotting the peak(s) of interest:

Utilises the above function cutdata to isolate the region of the data set to be inspected, allows the user to restrict the x_range plotted so it is more clearly displayed and allows the peak to be exported as a .png file if desired. As above it will open the plot in a new broswer tab by default.


In [ ]:
###############################################################################
# TO PLOT THE PEAK FOR INSPECTION # 
###############################################################################

# Utilising the function defined above we cut the dataset to allow us to inspect
# any peak of interest, saves the subset as xc and yc - as in, cut_x and cut_y
xc, yc = cutdata(x_start, x_end, x, y)

# As above, filename can be used to specify the region of interest
output_file('FILENAME.html')

p2 = figure(title = "PEAK OF INTEREST - Wavelength *INSERT WAVELENGTH HERE* \u212B",
            x_axis_label = 'Wavelength - \u212B',
            y_axis_label = 'Spectral flux density - 10\u207B\u00B9\u2077 erg s\u207B\u00B9 cm\u207B\u00B2 \u212B\u207B\u00B9')

# Dictates the X values displayed on the plot
p2.x_range=Range1d(x_start, x_end)

# Plots the subset of the data as a line graph
p2.line(xc, yc) 

# As above, dictates legend position
p2.legend.location = "top_right"

# As above
p2.add_tools(HoverTool(mode='vline')) 

# If you wish to save the file as a .png uncomment the line below:
#export_png(p2, filename="FILENAME.png")

# As above, except this shows the peak of interest plot
show(p2) 

To fit the peak we first need to model the curve, in this case we are using a Gaussian model for each peak.

A Gaussian model is defined by the following equation:
Aexp(−(x−x0)2(2σ2))

Where A
is its amplitude, x0 its position on the x axis, and σ a measure of its width.

A Gaussian curve of this form is frequently used to model and to fit peaks in spectra. The model involves a number of parameters (for the Gaussian curve these are A, x0 and σ). The model function will be called repeatedly by curve_fit() as it optimises the values of these three parameters. A list of initial values for these three parameters (p0) is provided as an extra input to the curve_fit() function.

These initial approximate parameters are used by the curve_fit() function to use as a starting point for the optimised values. This is optional but can be useful if the dataset is complex as it often improves the quality and speed of the fit.

In [ ]:
def GaussModel(x, a, x0, sigma):
    y_gauss = a*np.exp(-(x-x0)**2/(2*sigma**2))
    return y_gauss

# This is used to model the cut data so xc and yc are used in lieu of x and y

# A list of initial values for the three parameters of the Gaussian
# replace a, x0 and sigma with approximate values

p0 = [a, x0, sigma]

popt, pcov = curve_fit(GaussModel, xc, yc, p0)

# popt and pcov contain the optimised parameters and error values, uncomment to print 

# print(popt)
# print(pcov)

# For readibility, extract these values into named variables
a_fit     = popt[0]
x0_fit    = popt[1]
fwhm_fit = popt[2]

# perr calculates the errors on the returned parameters 
# Squares of the error values are on the diagonal of the covariance matrix
perr = np.sqrt(np.diag(pcov)) 

# For readibility, extract values from perr into named variables
a_err     = perr[0]
x0_err    = perr[1]
fwhm_err = perr[2]

# Prints fit parameters (to 6 s.f) and associated error estimates (to 6 s.f), in line with the notebook,
# where a = peak height, x0 = peak position, FWHM = Full Width Half Mean (2 * \sqrt{2*ln{2}} * sigma)

print()
print('fit parameters with error estimates')
print('***************************************************')
print(f'A     = {a_fit: .6g} +/- {a_err: .6g}')
print(f'x0    = {x0_fit: .6g} +/- {x0_err: .6g}')
print(f'fwhm = {fwhm_fit: .6g} +/- {fwhm_err: .6g}')
print('***************************************************')

# The following plots the data points and the optimised modelled curve

p3 = figure(title = "Peak of interest - *insert species*", 
            x_axis_label = 'Wavelength - \u212B', 
            y_axis_label = 'Spectral flux density - 10\u207B\u00B9\u2077 erg s\u207B\u00B9 cm\u207B\u00B2 \u212B\u207B\u00B9')

p3.line(xc, yc) 

p3.line(xc, GaussModel(xc, a_fit, x0_fit, fwhm_fit), 
        color = "red")

p3.add_tools(HoverTool(mode='vline'))

# If you wish to save the file as a .png uncomment the line below:
#export_png(p3, filename="FILENAME.png")

show(p3)